In [154]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.linear_model import Ridge
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso

## Kobe Bryant  shot selection

Cсылка на соревнование: https://www.kaggle.com/c/kobe-bryant-shot-selection

Goal: Fun and education

Using 20 years of data on Kobe's swishes and misses, can you predict which shots will find the bottom of the net? This competition is well suited for practicing classification basics, feature engineering, and time series analysis. Practice got Kobe an eight-figure contract and 5 championship rings. What will it get you?

This data contains the location and circumstances of every field goal attempted by Kobe Bryant took during his 20-year career. Your task is to predict whether the basket went in (shot_made_flag).

We have removed 5000 of the shot_made_flags (represented as missing values in the csv file). These are the test set shots for which you must submit a prediction. You are provided a sample submission file with the correct shot_ids needed for a valid prediction.

In [3]:
data = pd.read_csv('kobe.csv')

In [11]:
data

,action_type,combined_shot_type,game_event_id,game_id,lat,loc_x,loc_y,lon,minutes_remaining,period,...,shot_type,shot_zone_area,shot_zone_basic,shot_zone_range,team_id,team_name,game_date,matchup,opponent,shot_id
0,Jump Shot,Jump Shot,10,20000012,33.9723,167,72,-118.1028,10,1,...,2PT Field Goal,Right Side(R),Mid-Range,16-24 ft.,1610612747,Los Angeles Lakers,2000-10-31,LAL @ POR,POR,1
1,Jump Shot,Jump Shot,12,20000012,34.0443,-157,0,-118.4268,10,1,...,2PT Field Goal,Left Side(L),Mid-Range,8-16 ft.,1610612747,Los Angeles Lakers,2000-10-31,LAL @ POR,POR,2
2,Jump Shot,Jump Shot,35,20000012,33.9093,-101,135,-118.3708,7,1,...,2PT Field Goal,Left Side Center(LC),Mid-Range,16-24 ft.,1610612747,Los Angeles Lakers,2000-10-31,LAL @ POR,POR,3
3,Jump Shot,Jump Shot,43,20000012,33.8693,138,175,-118.1318,6,1,...,2PT Field Goal,Right Side Center(RC),Mid-Range,16-24 ft.,1610612747,Los Angeles Lakers,2000-10-31,LAL @ POR,POR,4
4,Driving Dunk Shot,Dunk,155,20000012,34.0443,0,0,-118.2698,6,2,...,2PT Field Goal,Center(C),Restricted Area,Less Than 8 ft.,1610612747,Los Angeles Lakers,2000-10-31,LAL @ POR,POR,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30692,Jump Shot,Jump Shot,397,49900088,33.9963,1,48,-118.2688,6,4,...,2PT Field Goal,Center(C),In The Paint (Non-RA),Less Than 8 ft.,1610612747,Los Angeles Lakers,2000-06-19,LAL vs. IND,IND,30693
30693,Tip Shot,Tip Shot,398,49900088,34.0443,0,0,-118.2698,6,4,...,2PT Field Goal,Center(C),Restricted Area,Less Than 8 ft.,1610612747,Los Angeles Lakers,2000-06-19,LAL vs. IND,IND,30694
30694,Running Jump Shot,Jump Shot,426,49900088,33.8783,-134,166,-118.4038,3,4,...,2PT Field Goal,Left Side Center(LC),Mid-Range,16-24 ft.,1610612747,Los Angeles Lakers,2000-06-19,LAL vs. IND,IND,30695
30695,Jump Shot,Jump Shot,448,49900088,33.7773,31,267,-118.2388,2,4,...,3PT Field Goal,Center(C),Above the Break 3,24+ ft.,1610612747,Los Angeles Lakers,2000-06-19,LAL vs. IND,IND,30696


In [12]:
data.describe()

,game_event_id,game_id,lat,loc_x,loc_y,lon,minutes_remaining,period,playoffs,seconds_remaining,shot_distance,shot_made_flag,team_id,shot_id
count,30697.000000,3.069700e+04,30697.000000,30697.000000,30697.000000,30697.000000,30697.000000,30697.000000,30697.000000,30697.000000,30697.000000,25697.000000,3.069700e+04,30697.000000
mean,249.190800,2.476407e+07,33.953192,7.110499,91.107535,-118.262690,4.885624,2.519432,0.146562,28.365085,13.437437,0.446161,1.610613e+09,15349.000000
std,150.003712,7.755175e+06,0.087791,110.124578,87.791361,0.110125,3.449897,1.153665,0.353674,17.478949,9.374189,0.497103,0.000000e+00,8861.604943
min,2.000000,2.000001e+07,33.253300,-250.000000,-44.000000,-118.519800,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.610613e+09,1.000000
25%,110.000000,2.050008e+07,33.884300,-68.000000,4.000000,-118.337800,2.000000,1.000000,0.000000,13.000000,5.000000,0.000000,1.610613e+09,7675.000000
50%,253.000000,2.090035e+07,33.970300,0.000000,74.000000,-118.269800,5.000000,3.000000,0.000000,28.000000,15.000000,0.000000,1.610613e+09,15349.000000
75%,368.000000,2.960047e+07,34.040300,95.000000,160.000000,-118.174800,8.000000,3.000000,0.000000,43.000000,21.000000,1.000000,1.610613e+09,23023.000000
max,659.000000,4.990009e+07,34.088300,248.000000,791.000000,-118.021800,11.000000,7.000000,1.000000,59.000000,79.000000,1.000000,1.610613e+09,30697.000000


In [13]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30697 entries, 0 to 30696
Data columns (total 25 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   action_type         30697 non-null  object 
 1   combined_shot_type  30697 non-null  object 
 2   game_event_id       30697 non-null  int64  
 3   game_id             30697 non-null  int64  
 4   lat                 30697 non-null  float64
 5   loc_x               30697 non-null  int64  
 6   loc_y               30697 non-null  int64  
 7   lon                 30697 non-null  float64
 8   minutes_remaining   30697 non-null  int64  
 9   period              30697 non-null  int64  
 10  playoffs            30697 non-null  int64  
 11  season              30697 non-null  object 
 12  seconds_remaining   30697 non-null  int64  
 13  shot_distance       30697 non-null  int64  
 14  shot_made_flag      25697 non-null  float64
 15  shot_type           30697 non-null  object 
 16  shot

In [5]:
target = 'shot_made_flag'

**Задания:**

1. Провести анализ данных. Много хороших примеров анализа можно посмотреть здесь https://www.kaggle.com/c/kobe-bryant-shot-selection/kernels
2. Подготовить фичи для обучения модели - нагенерить признаков, обработать пропущенные значения, проверить на возможные выбросы, обработать категориальные признаки и др.
3. Обучить линейную модель, Lasso, Ridge (с встроеной регуляризацией) на тех же признаках - построить сравнительную таблицу или график коэффициентов, сделать заключения о том, как меняется величина коэффициентов, какие зануляются. 

**Дополнительно**
4. Сравнить результаты на тестовом наборе данных - сделать train_test_split в самом начале, подготовить переменные, сравнить результаты работы классификаторов (те же 3), метрика ROC AUC
5. Построить PCA на подготовленных признаках, посмотреть, какие компоненты составляют наибольшую часть дисперсии целевой переменной

In [178]:
data.nunique()

action_type              57
combined_shot_type        6
game_event_id           620
game_id                1559
lat                     457
loc_x                   489
loc_y                   457
lon                     489
minutes_remaining        12
period                    7
playoffs                  2
season                   20
seconds_remaining        60
shot_distance            74
shot_made_flag            2
shot_type                 2
shot_zone_area            6
shot_zone_basic           7
shot_zone_range           5
team_id                   1
team_name                 1
game_date              1559
matchup                  74
opponent                 33
shot_id               30697
dtype: int64

In [179]:
data.shot_zone_range.unique()

array(['16-24 ft.', '8-16 ft.', 'Less Than 8 ft.', '24+ ft.',
       'Back Court Shot'], dtype=object)

In [180]:
data_clear = data.drop(['team_id', 'team_name'], axis=1) #убираем колонки team_id и team_name так как в них всего одно значение соответственно они вряд ли будут оказывать влияние на предсказание.

In [181]:
data_clear.drop(['game_event_id', 'game_id'], axis=1, inplace=True) #номер игры и номер события вряд ли оказывают влияние на результат игры, поэтому также убираем

In [182]:
data_clear["game_date"] = pd.to_datetime(data_clear["game_date"]) #Преотбразуем дату игры, разбив ее на год, месяц и день, не удаляем эти данные, так как от них может зависеть то в какой форме игрок. Хотя можно удалить и даты и ориентироваться на сезон.
data_clear["game_year"] = data_clear["game_date"].dt.year
data_clear["game_month"] = data_clear["game_date"].dt.month
data_clear["game_day"] = data_clear["game_date"].dt.day
data_clear.drop(['game_date',], axis=1, inplace=True)

In [183]:
data_clear["home_play"] = data_clear["matchup"].str.contains("vs").astype("int") #Если верить описанию что колонка mathup означает где и с кем была игра, то преобразуем ее в бинарный признак дома или в гостях была игра, так как с кем была игра есть в колонке opponent
data_clear = data_clear.drop("matchup", axis=1)

In [184]:
data_clear.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30697 entries, 0 to 30696
Data columns (total 23 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   action_type         30697 non-null  object 
 1   combined_shot_type  30697 non-null  object 
 2   lat                 30697 non-null  float64
 3   loc_x               30697 non-null  int64  
 4   loc_y               30697 non-null  int64  
 5   lon                 30697 non-null  float64
 6   minutes_remaining   30697 non-null  int64  
 7   period              30697 non-null  int64  
 8   playoffs            30697 non-null  int64  
 9   season              30697 non-null  object 
 10  seconds_remaining   30697 non-null  int64  
 11  shot_distance       30697 non-null  int64  
 12  shot_made_flag      25697 non-null  float64
 13  shot_type           30697 non-null  object 
 14  shot_zone_area      30697 non-null  object 
 15  shot_zone_basic     30697 non-null  object 
 16  shot

In [185]:
data_clear

,action_type,combined_shot_type,lat,loc_x,loc_y,lon,minutes_remaining,period,playoffs,season,...,shot_type,shot_zone_area,shot_zone_basic,shot_zone_range,opponent,shot_id,game_year,game_month,game_day,home_play
0,Jump Shot,Jump Shot,33.9723,167,72,-118.1028,10,1,0,2000-01,...,2PT Field Goal,Right Side(R),Mid-Range,16-24 ft.,POR,1,2000,10,31,0
1,Jump Shot,Jump Shot,34.0443,-157,0,-118.4268,10,1,0,2000-01,...,2PT Field Goal,Left Side(L),Mid-Range,8-16 ft.,POR,2,2000,10,31,0
2,Jump Shot,Jump Shot,33.9093,-101,135,-118.3708,7,1,0,2000-01,...,2PT Field Goal,Left Side Center(LC),Mid-Range,16-24 ft.,POR,3,2000,10,31,0
3,Jump Shot,Jump Shot,33.8693,138,175,-118.1318,6,1,0,2000-01,...,2PT Field Goal,Right Side Center(RC),Mid-Range,16-24 ft.,POR,4,2000,10,31,0
4,Driving Dunk Shot,Dunk,34.0443,0,0,-118.2698,6,2,0,2000-01,...,2PT Field Goal,Center(C),Restricted Area,Less Than 8 ft.,POR,5,2000,10,31,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30692,Jump Shot,Jump Shot,33.9963,1,48,-118.2688,6,4,1,1999-00,...,2PT Field Goal,Center(C),In The Paint (Non-RA),Less Than 8 ft.,IND,30693,2000,6,19,1
30693,Tip Shot,Tip Shot,34.0443,0,0,-118.2698,6,4,1,1999-00,...,2PT Field Goal,Center(C),Restricted Area,Less Than 8 ft.,IND,30694,2000,6,19,1
30694,Running Jump Shot,Jump Shot,33.8783,-134,166,-118.4038,3,4,1,1999-00,...,2PT Field Goal,Left Side Center(LC),Mid-Range,16-24 ft.,IND,30695,2000,6,19,1
30695,Jump Shot,Jump Shot,33.7773,31,267,-118.2388,2,4,1,1999-00,...,3PT Field Goal,Center(C),Above the Break 3,24+ ft.,IND,30696,2000,6,19,1


In [186]:
#Находим категориальные признаки
cat_feat = list(data_clear.dtypes[data_clear.dtypes == object].index)

In [187]:
# Создаем дамми-переменную для категорий
dummy_data = pd.get_dummies(data_clear[cat_feat], columns=cat_feat)

In [188]:
#Объединяем дамми переменную и основной массив и удаляем изначальные категориальные признаки
data_clear.drop(cat_feat, axis=1, inplace=True)
data_clear=pd.concat([data_clear, dummy_data], axis=1)

In [189]:
data_clear

,lat,loc_x,loc_y,lon,minutes_remaining,period,playoffs,seconds_remaining,shot_distance,shot_made_flag,...,opponent_PHI,opponent_PHX,opponent_POR,opponent_SAC,opponent_SAS,opponent_SEA,opponent_TOR,opponent_UTA,opponent_VAN,opponent_WAS
0,33.9723,167,72,-118.1028,10,1,0,27,18,NaN,...,0,0,1,0,0,0,0,0,0,0
1,34.0443,-157,0,-118.4268,10,1,0,22,15,0.0,...,0,0,1,0,0,0,0,0,0,0
2,33.9093,-101,135,-118.3708,7,1,0,45,16,1.0,...,0,0,1,0,0,0,0,0,0,0
3,33.8693,138,175,-118.1318,6,1,0,52,22,0.0,...,0,0,1,0,0,0,0,0,0,0
4,34.0443,0,0,-118.2698,6,2,0,19,0,1.0,...,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30692,33.9963,1,48,-118.2688,6,4,1,5,4,0.0,...,0,0,0,0,0,0,0,0,0,0
30693,34.0443,0,0,-118.2698,6,4,1,5,0,NaN,...,0,0,0,0,0,0,0,0,0,0
30694,33.8783,-134,166,-118.4038,3,4,1,28,21,1.0,...,0,0,0,0,0,0,0,0,0,0
30695,33.7773,31,267,-118.2388,2,4,1,10,26,0.0,...,0,0,0,0,0,0,0,0,0,0


In [190]:
#создаем тренировочный набор с данными по которым есть результат в колонке shot_made_flag
X_train=data_clear[data_clear.shot_made_flag.notna()]
y_train=X_train[['shot_made_flag']]
X_train.drop(['shot_made_flag'], axis=1, inplace=True)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [191]:
#создаем тренировочный набор с данными по которым нет результата в колонке shot_made_flag
X_test=data_clear[data_clear.shot_made_flag.isna()]
X_test.drop(['shot_made_flag',], axis=1, inplace=True)

Обучаем линейную модель, Lasso, Ridge

In [192]:
model_rg = Ridge()
model_lr = LinearRegression()
model_ls = Lasso()

In [193]:
model_rg.fit(X_train, y_train)

Ridge()

In [194]:
model_lr.fit(X_train, y_train)

LinearRegression()

In [195]:
model_ls.fit(X_train, y_train)

Lasso()

In [196]:
model_rg.predict(X_test)

array([[0.3784006 ],
       [0.31898694],
       [0.75994818],
       ...,
       [0.74635897],
       [0.81033761],
       [0.32286509]])

In [197]:
model_lr.predict(X_test)

array([[0.3844986 ],
       [0.32412338],
       [0.76451874],
       ...,
       [0.73628235],
       [0.80031586],
       [0.31235886]])

In [198]:
model_ls.predict(X_test)

array([0.46713096, 0.49813323, 0.51785975, ..., 0.46923021, 0.43751796,
       0.50304925])

In [230]:
comparison_coef =pd.concat([pd.DataFrame(model_rg.coef_[0], columns=['Ridge']), 
                            pd.DataFrame(model_lr.coef_[0], columns=['LinearRegression']), 
                            pd.DataFrame(model_ls.coef_, columns=['Lasso'])], axis=1 )

In [245]:
pd.set_option('display.max_rows', None)
comparison_coef

,Ridge,LinearRegression,Lasso
0,1.292389e-07,6.049827e+03,0.000000e+00
1,-6.485036e-05,-1.499247e+05,-0.000000e+00
2,-1.292386e-04,6.049689e+00,-7.046738e-04
3,-6.484974e-08,1.499247e+08,-0.000000e+00
4,3.200065e-03,3.195365e-03,0.000000e+00
5,-1.197039e-02,-1.179582e-02,-0.000000e+00
6,1.564129e-02,1.753497e-02,0.000000e+00
7,7.240158e-04,7.151960e-04,0.000000e+00
8,1.812137e-03,1.898332e-03,-0.000000e+00
9,-1.143967e-06,-8.940697e-07,-4.827886e-07


Для Ridge и Линейной регрессии почти все коэффициенты стремятся к нулю, но не отнуляются. Для Лассо - не обнуляются только два коэффициента - loc_y, shot_id 

Дополнительно 4. Сравнить результаты на тестовом наборе данных - сделать train_test_split в самом начале, подготовить переменные, сравнить результаты работы классификаторов (те же 3), метрика ROC AUC 5. Построить PCA на подготовленных признаках, посмотреть, какие компоненты составляют наибольшую часть дисперсии целевой переменной

Не очень понятно, зачем делать train_test_split, если по условиям задачи деление должно идти по колонке shot_made_flag, тестовая выборка, там где стоит NaN

"сравнить результаты работы классификаторов (те же 3), метрика ROC AUC 5" - каких тех же трех классификаторов, если использовались не классификаторы?? Если для Ridge есть аналогичный классификатор, для Линейной регрессии можно предположить, что надо использовать Логистическую регрессию, то для Лассо - я аналогов не нашел. ROC AUC - можно применить только для классификатора.

In [247]:
from sklearn.decomposition import PCA

In [248]:
pca = PCA(n_components=2)
pca.fit(X_train)

PCA(n_components=2)

In [249]:
print(pca.explained_variance_ratio_)

[9.99740122e-01 1.54348689e-04]


In [250]:
pca.fit_transform(X_test)

array([[ 1.54551846e+04, -1.49122594e+02],
       [ 1.54480713e+04,  1.61658309e+01],
       [ 1.54390753e+04,  1.66790577e+01],
       ...,
       [-1.52269731e+04,  6.31882486e+01],
       [-1.52309196e+04, -2.00848589e+01],
       [-1.52379164e+04, -5.57106084e+00]])

In [256]:
model_rg_pca = Ridge()
model_lr_pca = LinearRegression()
model_ls_pca = Lasso()

In [257]:
model_rg_pca.fit(pca.fit_transform(X_train), y_train)
model_lr_pca.fit(pca.fit_transform(X_train), y_train)
model_ls_pca.fit(pca.fit_transform(X_train), y_train)

Lasso()

In [258]:
model_rg_pca.predict(pca.fit_transform(X_test))

array([[0.43724186],
       [0.43462351],
       [0.43462193],
       ...,
       [0.45627755],
       [0.45760218],
       [0.45737692]])

In [259]:
model_lr_pca.predict(pca.fit_transform(X_test))

array([[0.43724186],
       [0.43462351],
       [0.43462193],
       ...,
       [0.45627755],
       [0.45760218],
       [0.45737692]])

In [260]:
model_ls_pca.predict(pca.fit_transform(X_test))

array([0.43507177, 0.43507688, 0.43508333, ..., 0.45708654, 0.45708937,
       0.45709439])

In [261]:
comparison_coef =pd.concat([pd.DataFrame(model_rg_pca.coef_[0], columns=['Ridge']), 
                            pd.DataFrame(model_lr_pca.coef_[0], columns=['LinearRegression']), 
                            pd.DataFrame(model_ls_pca.coef_, columns=['Lasso'])], axis=1 )

In [262]:
comparison_coef

,Ridge,LinearRegression,Lasso
0,-7.302485e-07,-7.302485e-07,-7.175104e-07
1,-1.587253e-05,-1.587253e-05,-0.000000e+00
